## Create input sequence and input embeddings

In [1]:
import torch
import torch.nn as nn

input_seq = ['<sos>', 'hello', 'world', '<eos>']
# imagine we have a vocabulary of 10000 words
vocab_size = 10000
embedding_dim = 64

vocab_embedding_table = nn.Embedding(vocab_size, embedding_dim)

# create input embeddings
input_embeddings = vocab_embedding_table(torch.LongTensor(input_seq))

ModuleNotFoundError: No module named 'torch'

## Create positional encodings

In [ ]:
max_seq_len = len(input_seq)
positional_encodings = nn.Embedding(max_seq_len, embedding_dim)

for i in range(max_seq_len):
    for j in range(embedding_dim):
        if j % 2 == 0:
            positional_encodings[i][j] = torch.sin(i / (10000 ** (j / embedding_dim)))
        else:
            positional_encodings[i][j] = torch.cos(i / (10000 ** ((j - 1) / embedding_dim)))

Add input embeddings and positional encodings

In [ ]:
input_embeddings = input_embeddings + positional_encodings